# 关联分析

关联分析：从大数据集中寻找物品间的隐含关系
频繁项集：经常出现在一起的物品集合
关联规则：暗示两种物品之间可能存在很强的关系
频繁的衡量尺度：
* 支持度：数据集中包含该项集记录所占比例
* 置信度：如{尿布}->{葡萄酒}的置信度为 `r = 支持度({尿布，葡萄酒}) / 支持度({尿布})`，
意味着对于包含{尿布}的记录，对其中的r * 记录数都适用

要想找到支持度大于0.8的项集，我们就需要对所有的物品进行排列组合得到所有可能的项集，再进行支持度的计算。
这会十分耗时低效，我们将分析Apriori原理，该原理可以减小关联学习的计算量。

## Apriori原理
>a priori —— 一个先验。在拉丁语中指“来自以前”。我们在贝叶斯统计时经常使用先验知识作为判断的条件，这些知识来自领域的知识，先前的测量结果等等。

原理内容：如果某个项集是频繁的，那么它所有的子集也是频繁的。同理，如果某个项集是非频繁集，那么它的所有超集也是非频繁的。

利用此原理可有效降低项集的指数级增长

## 用Apriori发现频繁集
算法描述：

    生成所有单个物品的项集列表
    对每个数据记录
        对每个项集
            包含该项集就增加总计数值
    对每个项集的的总数/总数据记录数
        如果满足最小支持度，则保留此项集
    对剩下的项集组合以生成包含两个元素的项集
    重复上述去除项集的操作，直到所有的项集删除

### 生成候选项集
* 加载数据集

In [1]:
def load_data():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]    

* 对 dataSet 进行去重，排序，放入 list 中，然后转换所有的元素为 frozenset

In [2]:
def create_c1(dataset):
    c1 = []
    for row in dataset:
        for item in row:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    return list(map(frozenset, c1))

* 计算候选数据集 CK 在数据集 D 中的支持度，并返回支持度大于最小支持度（minSupport）的数据

In [3]:
def scan(dataset, candidate, min_support):
    sscnt = {}
    for row in dataset:
        for can in candidate:
            if can.issubset(row):
                if can not in sscnt:
                    sscnt[can] = 1
                else:
                    sscnt[can] += 1
    num = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key]/num
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data

In [4]:
dataset = load_data()
c1 = create_c1(dataset)
print(c1)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]


In [5]:
D = list(map(set, dataset))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

### 使用0.5作为最小支持度

In [6]:
L1, supportdata0 = scan(D, c1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

发现4被删除，说明4没有达到最小支持度

## 组织完整Apriori算法
    
    当集合中的个数大于0时
        构建一个k个项组成的候选项集的列表
        检查数据以确认每个项集都是频繁的
        保留频繁项集并构建k+1项组成的候选项集的列表